# Bring Your Own Checkpoint (BYOC) to SamabaStudio 

Example checkpoint download from Huggingface

In [ ]:
import os
import sys
current_dir = os.getcwd()
utils_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(utils_dir, ".."))

sys.path.append(utils_dir)
sys.path.append(repo_dir)

from huggingface_hub import hf_hub_download, HfApi

model_name = "NousResearch/Hermes-2-Theta-Llama-3-8B"  # Replace with your desired model

target_dir = os.path.join(current_dir, "models")  
os.makedirs(target_dir, exist_ok=True)

repo_files = HfApi().list_repo_files(model_name)

for file_name in repo_files:
    hf_hub_download(repo_id=model_name, filename=file_name, cache_dir=target_dir)

After download, modify [config.yaml](./config.yaml) file with the snapshot of the model and required parameters

check at the `chat_template` key in the [tokenizer_config.json](./models/models--NousResearch--Hermes-2-Theta-Llama-3-8B/snapshots/57a73110702e7b05ba3f39fef36297454c680725/tokenizer_config.json) file inside the model snapshot and modify it to follow a chat jinja template like this:

```json
  "chat_template": "{% for message in messages %}{% set content = '<|im_start|>' + message['role'] + '\n' + message['content'] | trim + '<|im_end|>'+'\n' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}<|im_start|>assistant\n"
```

In [ ]:
from dedicated_env.snsdk_wrapper import SnsdkWrapper
byoc = SnsdkWrapper("./config.yaml")

Check your jinja chat template

In [ ]:
test_messages = [
    {"role": "system", "content": "This is a system prompt."},
    {"role": "user", "content": "This is a user prompt."},
    {"role": "assistant", "content": "This is a response from the assistant."},
    {"role": "user", "content": "This is an user follow up"}
    ]
byoc.check_chat_templates(test_messages)

Set BYOC object with checkpoints to upload config

Get params from checkpoints paths set in [config.yaml](./config.yaml) file

In [ ]:
byoc.find_config_params(update_config_file=True)

Find suitable SambaStudio apps for your checkpoints

In [ ]:
byoc.get_suitable_apps()

Set in the `app_id` key of your checkpoint in the [config.yaml](./config.yaml) file the desired app_id of your checkpoints

Upload the checkpoints in your [config.yaml](./config.yaml) file to SambaStudio (this could take from minutes to hours depending on the size of your checkpoints)

In [ ]:
#load the updated config.yaml file including app ids and model params
byoc = SnsdkWrapper("./config.yaml")

In [ ]:
byoc.upload_checkpoints()

You can check the status of the checkpoints after the upload process

In [ ]:
byoc.get_checkpoints_status()

We can create a bundle model including uploaded and existing checkpoints in sambastudio setting `composite_model` params in [config.yaml](./config.yaml) file

In [ ]:
byoc.create_composite_model()

Then we can deploy the bundle model, setting `deployement` params in [config.yaml](./config.yaml) file

In [ ]:
# create project
byoc.create_project()

In [ ]:
# deploy your model in an endpoint
byoc.create_endpoint()

In [ ]:
# get endpoint details
byoc.get_endpoint_details()

Use the models

In [ ]:
from langchain_sambanova import ChatSambaStudio

endpoint_details = byoc.get_endpoint_details()
endpoint_url = endpoint_details["langchain_wrapper_env"].get("SAMBASTUDIO_URL")
endpoint_api_key = endpoint_details["langchain_wrapper_env"].get("SAMBASTUDIO_API_KEY")

llm = ChatSambaStudio(
    sambastudio_url=endpoint_url,
    sambastudio_api_key=endpoint_api_key,
    do_sample= True, 
    temperature= 0.01,
    max_tokens= 512,
    top_p= 1.0,
    top_k= 50,
    process_prompt=False,
    model= "HermesProInstructV10"
)
print(llm.invoke("tell me a joke").content)